# Merge Spectractor Results with Exposures list from Butler (saved in csv file)

- author : Sylvie Dagoret-Campagne
- creation date : 2025-09-20
- last update : 2025-09-21  add  imports from BUTLER00_parameters

- read the list of exposure generated by TOOL_ListofExposures.ipynb
- plot the different filters vs time

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"


In [ ]:
from BUTLER00_parameters import *

## Configuration

In [ ]:
FLAG_REPO_EMBARGO = True

## Spectractor Results

In [ ]:
#version_results = "v2"
#legendtag = {"v1" : "v3.1.0","v2":"v3.1.0"}

#atmfilenamesdict = {"v1":"../2025-06-26-SpectractorExtraction-FromButler/data/spectro/auxtel_atmosphere_20250912a_repomain_v1.npy",
#                    "v2":"../2025-06-26-SpectractorExtraction-FromButler/data/spectro/auxtel_run_20250917_w_2025_25_spectractorv31_holoallfilt_a_repomain_v1.npy",}


#atmfilename = atmfilenamesdict[version_results]
#tag = legendtag[version_results] 


In [ ]:
atmfilename = extractedfilesdict["run_v4"]
tag = legendtag["run_v4"] 

In [ ]:
print(f"Spectractor Extracted atmospheric parameters file : {atmfilename} ")

In [ ]:
input_path = os.path.dirname(atmfilename)
inputfilename = os.path.basename(atmfilename)
inputfilename_root = inputfilename.split(".")[0] 
inputfilename_ext = inputfilename.split(".")[-1] 

output_path = input_path + '_merged'
outputfilename = inputfilename_root + '_merged' + '.' + inputfilename_ext
#outputfilename = inputfilename_root + '_merged' + '.npz'

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)

In [ ]:
df_spec

## List of Exposures

In [ ]:
if FLAG_REPO_EMBARGO:
    repo="/repo/embargo"
else:
    repo="/repo/main"
reponame = repo.replace("/","_")

path_exposureslist = "data/butlerregistry"
listexposures_file = os.path.join(path_exposureslist,"holosummary_all_filters" + reponame + ".csv")

In [ ]:
df_exp = pd.read_csv(listexposures_file,index_col=0)
df_exp = df_exp.reset_index(drop=True)

In [ ]:
df_exp

In [ ]:
df_exp_prefixed = df_exp.rename(
    columns={col: f"ex_{col}" for col in df_exp.columns if col != "id"}
)

In [ ]:
df_exp_prefixed 

## Merge

In [ ]:
# Merge sur la colonne 'id'
df_spec_merged = pd.merge(df_spec, df_exp_prefixed, on="id", how="inner")

In [ ]:
df_spec_merged

## Save output

In [ ]:
os.makedirs(output_path, exist_ok=True)

In [ ]:
print(outputfilename) 

In [ ]:
outputfullfilename = os.path.join(output_path,outputfilename)

In [ ]:
rec_array = df_spec_merged.to_records(index=False)

In [ ]:
np.save(outputfullfilename , rec_array)

## Check

In [ ]:
newspecdata = np.load(outputfullfilename,allow_pickle=True)

In [ ]:
#newspecdata

In [ ]:
df_newspec = pd.DataFrame(newspecdata)

In [ ]:
df_newspec